In [1]:
%load_ext rpy2.ipython
from IPython.display import FileLink

In [2]:
%%R
library(phyloseq)
library(dplyr); library(tidyr)
library(ggplot2)
library(gridExtra)
library(foreach)
library(doParallel)


Attaching package: ‘dplyr’

The following object is masked from ‘package:stats’:

    filter

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: grid
foreach: simple, scalable parallel programming from Revolution Analytics
Use Revolution R for scalability, fault tolerance and more.
http://www.revolutionanalytics.com
Loading required package: iterators
Loading required package: parallel


###These files are from the [l2fc sparsity notebook](./Sparsity_make_l2fc_table.ipynb) and [l2fc no sparsity notebook](./l2fc_table_nospar.ipynb)

In [3]:
%%R
l2fc = read.csv("/var/seq_data/priming_exp/data/l2fc_table.csv")

In [4]:
%%R
head(l2fc)

  sparsity_threshold Day Treatment       OTU log2FoldChange         p    Rank1
1               0.25  14    13C000  OTU.4204    -0.20553170 1.0000000 Bacteria
2               0.25  14    13C000   OTU.467    -0.33054855 1.0000000 Bacteria
3               0.25  14    13C000   OTU.905     0.01698885 0.9827594 Bacteria
4               0.25  14    13C000  OTU.7253     0.02799601 0.9772968 Bacteria
5               0.25  14    13C000    OTU.67    -0.60115954 1.0000000 Bacteria
6               0.25  14    13C000 OTU.10580     0.36193427 0.7248674 Bacteria
          Rank2 Rank3                Rank4 Rank5 Rank6 Rank7 Rank8 padj
1 Acidobacteria DA023                 <NA>  <NA>  <NA>  <NA>  <NA>    1
2 Acidobacteria DA023 uncultured_bacterium  <NA>  <NA>  <NA>  <NA>    1
3 Acidobacteria DA023 uncultured_bacterium  <NA>  <NA>  <NA>  <NA>    1
4 Acidobacteria DA023 uncultured_bacterium  <NA>  <NA>  <NA>  <NA>    1
5 Acidobacteria DA023 uncultured_bacterium  <NA>  <NA>  <NA>  <NA>    1
6 Acidobacteria

In [5]:
%%R
FDR = 0.10

df = l2fc %>%
    group_by(OTU, Treatment, Rank2) %>%
    summarize(minP = min(padj)) %>%
    filter(minP <= FDR) 

df

Source: local data frame [1,000 x 4]
Groups: OTU, Treatment

         OTU Treatment          Rank2         minP
1      OTU.1    13C000 Actinobacteria 1.323119e-02
2     OTU.10    13C000 Proteobacteria 8.989932e-06
3     OTU.10    13C100 Proteobacteria 3.352715e-05
4     OTU.10    13C700 Proteobacteria 5.974757e-13
5   OTU.1009    13C000  Bacteroidetes 5.712882e-02
6   OTU.1009    13C100  Bacteroidetes 1.484723e-04
7   OTU.1009    13C700  Bacteroidetes 2.958242e-07
8    OTU.101    13C000 Actinobacteria 2.899518e-02
9   OTU.1016    13C700  Acidobacteria 2.081194e-03
10 OTU.10167    13C000  Acidobacteria 1.087521e-03
..       ...       ...            ...          ...


In [9]:
%%R
physeq = import_biom("/var/seq_data/priming_exp/data/otu_table.biom", "/var/seq_data/priming_exp/data/otusn.tree")
sample.data = import_qiime_sample_data("/var/seq_data/priming_exp/data/allsample_metadata_nomock.txt")
physeq = merge_phyloseq(physeq,sample.data)
physeq.md = sample_data(physeq)
physeq.PE = prune_samples((physeq.md$Bulk == 0), physeq)
physeq.PE.snorm = transform_sample_counts(physeq.PE, function(x) x/sum(x))
physeq.PE.thresh = filter_taxa(physeq.PE.snorm, function(x) sum(x > 0) > (0.25 * length(x)), TRUE)
#physeq.PE.responders = subset_taxa(physeq.PE.thresh, OTU == "OTU.1", TRUE)
#print(physeq.PE.responders)
 print(physeq.PE.thresh)   

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 3203 taxa and 395 samples ]
sample_data() Sample Data:       [ 395 samples by 13 sample variables ]
phy_tree()    Phylogenetic Tree: [ 3203 tips and 3202 internal nodes ]
